In [41]:
import pandas as pd
from citipy import citipy
import numpy as np
import csv
from datetime import datetime
import json
from pandas.io.json import json_normalize

#import data
weather = pd.read_json("ow.json") #1/1/2013-1/1/2020 hourly weather data
weather = weather.dropna(how='any')
crime = pd.read_csv("crime.csv")
weather

,city_name,lat,lon,main,wind,clouds,weather,dt,dt_iso,timezone,snow,rain
1530,Denver,39.739236,-104.984862,"{'temp': 34.12, 'temp_min': 26.01, 'temp_max':...","{'speed': 9.17, 'deg': 130}",{'all': 1},"[{'id': 500, 'main': 'Rain', 'description': 'l...",1362506400,2013-03-05 18:00:00 +0000 UTC,-25200,{'24h': 0.76},"{'1h': 0.51, '24h': 0.51}"
1531,Denver,39.739236,-104.984862,"{'temp': 35.49, 'temp_min': 28, 'temp_max': 43...","{'speed': 6.93, 'deg': 110}",{'all': 1},"[{'id': 800, 'main': 'Clear', 'description': '...",1362510000,2013-03-05 19:00:00 +0000 UTC,-25200,{'24h': 0.76},{'24h': 0.71}
1532,Denver,39.739236,-104.984862,"{'temp': 36.23, 'temp_min': 30.99, 'temp_max':...","{'speed': 9.17, 'deg': 300}",{'all': 1},"[{'id': 800, 'main': 'Clear', 'description': '...",1362513600,2013-03-05 20:00:00 +0000 UTC,-25200,{'24h': 0.76},{'24h': 0.71}
1539,Denver,39.739236,-104.984862,"{'temp': 30.04, 'temp_min': 26.01, 'temp_max':...","{'speed': 3.36, 'deg': 0}",{'all': 1},"[{'id': 800, 'main': 'Clear', 'description': '...",1362538800,2013-03-06 03:00:00 +0000 UTC,-25200,{'24h': 0.76},{'24h': 1.27}
1621,Denver,39.739236,-104.984862,"{'temp': 32.56, 'temp_min': 30.92, 'temp_max':...","{'speed': 9.17, 'deg': 330}",{'all': 90},"[{'id': 500, 'main': 'Rain', 'description': 'l...",1362834000,2013-03-09 13:00:00 +0000 UTC,-25200,"{'1h': 0.25, '24h': 0.25}","{'1h': 0.51, '24h': 0.51}"
...,...,...,...,...,...,...,...,...,...,...,...,...
61321,Denver,39.739236,-104.984862,"{'temp': 31.46, 'temp_min': 28.99, 'temp_max':...","{'speed': 8.05, 'deg': 130}",{'all': 20},"[{'id': 801, 'main': 'Clouds', 'description': ...",1577732400,2019-12-30 19:00:00 +0000 UTC,-25200,{'24h': 0.7000000000000001},{'24h': 1.27}
61324,Denver,39.739236,-104.984862,"{'temp': 32.92, 'temp_min': 30.2, 'temp_max': ...","{'speed': 10.29, 'deg': 130}",{'all': 20},"[{'id': 500, 'main': 'Rain', 'description': 'l...",1577743200,2019-12-30 22:00:00 +0000 UTC,-25200,{'24h': 0.25},"{'1h': 0.25, '24h': 0.51}"
61345,Denver,39.739236,-104.984862,"{'temp': 34.47, 'temp_min': 30.99, 'temp_max':...","{'speed': 12.75, 'deg': 190}",{'all': 0},"[{'id': 800, 'main': 'Clear', 'description': '...",1577815200,2019-12-31 18:00:00 +0000 UTC,-25200,{'24h': 0.51},{'24h': 0.76}
61351,Denver,39.739236,-104.984862,"{'temp': 35.76, 'temp_min': 32, 'temp_max': 39...","{'speed': 6.93, 'deg': 200}",{'all': 0},"[{'id': 500, 'main': 'Rain', 'description': 'l...",1577836800,2020-01-01 00:00:00 +0000 UTC,-25200,{},"{'1h': 0.25, '24h': 1.65}"


In [30]:
drop_list = ['GEO_X', 'GEO_Y', 'DISTRICT_ID', 
             'REPORTED_DATE', 'LAST_OCCURRENCE_DATE',
            'INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE_EXTENSION']
crime.drop(columns=drop_list, inplace=True)
crime[['Date','Hour', 'am/pm']] = crime.FIRST_OCCURRENCE_DATE.str.split(expand=True)

In [31]:
crime = crime.rename(columns = {'OFFENSE_CODE':'Code', 
                               'OFFENSE_TYPE_ID':'Offense_Type', 
                               'INCIDENT_ADDRESS':'Address', 
                               'GEO_LON':'Lon',
                               'GEO_LAT':'Lat', 
                               'PRECINCT_ID':'Precint', 
                               'NEIGHBORHOOD_ID':'Neighborhood'})

In [38]:
#expand list from columns into new databases
weather_temp = (json_normalize(weather['main']))
weather_wind = json_normalize(weather['wind'])
weather_clouds = json_normalize(weather['clouds'])

In [39]:
#create new columns for desired fields from expanded list
weather_temp.head()
weather['Temp'] = weather_temp['temp']
weather['Temp Max'] = weather_temp['temp_max']
weather['Wind'] = weather_wind.speed
weather['Clouds'] = weather_clouds
#split date iso into columns, then delete unwanted columns 
weather[['Date','Hour', 'Trash', 'Trash2']] = weather.dt_iso.str.split(expand=True)
columns = ['wind', 'clouds', 'weather','main', 'timezone', 'snow', 'rain', 'dt', 'Trash', 'Trash2', 'dt_iso']
weather.drop(columns=columns, inplace=True)

In [40]:
#create dataframe for speends over 40mph 
wind_df = weather.loc[weather.Wind > 40]
wind_df

,city_name,lat,lon,Temp,Temp Max,Wind,Clouds,Date,Hour


In [35]:
#create dataframe for temps under 10f 
cold_df = weather.loc[weather['Temp Max'] < 10]
cold_df_new = cold_df.set_index('Date')
cold_df_new

,city_name,lat,lon,Temp,Temp Max,Wind,Clouds,Hour
Date,,,,,,,,
2013-01-12,Denver,39.739236,-104.984862,4.10,8.60,9.17,40,08:00:00
2013-01-12,Denver,39.739236,-104.984862,5.90,8.60,8.05,75,09:00:00
2013-01-12,Denver,39.739236,-104.984862,4.10,6.80,5.82,40,10:00:00
2013-01-12,Denver,39.739236,-104.984862,5.00,5.00,3.36,1,11:00:00
2013-01-12,Denver,39.739236,-104.984862,4.10,5.00,3.36,90,12:00:00
...,...,...,...,...,...,...,...,...
2019-10-30,Denver,39.739236,-104.984862,6.73,9.00,4.70,0,12:00:00
2019-10-30,Denver,39.739236,-104.984862,6.42,9.00,3.36,0,13:00:00
2019-10-30,Denver,39.739236,-104.984862,6.17,8.01,3.36,100,14:00:00


In [20]:
crime = crime.dropna(how='any')
crime

,Code,Offense_Type,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,Address,Lon,Lat,Precint,Neighborhood,IS_CRIME,IS_TRAFFIC,Date,Hour,am/pm
2,2305,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,2932 S JOSEPHINE ST,-104.957381,39.663490,314,wellshire,1,0,3/4/2016,8:00:00,PM
3,2399,theft-other,larceny,1/30/2018 7:20:00 PM,705 S COLORADO BLVD,-104.941440,39.702698,312,belcaro,1,0,1/30/2018,7:20:00,PM
4,2303,theft-shoplift,larceny,6/22/2017 8:53:00 PM,2810 E 1ST AVE,-104.955370,39.717107,311,cherry-creek,1,0,6/22/2017,8:53:00,PM
5,5499,traf-other,all-other-crimes,1/31/2018 12:44:00 AM,2100 BLOCK E 17TH AVE,-104.961928,39.743149,622,city-park-west,1,0,1/31/2018,12:44:00,AM
6,2304,theft-parts-from-vehicle,theft-from-motor-vehicle,6/1/2017 12:15:00 PM,995 N FEDERAL BLVD,-105.025543,39.732790,122,villa-park,1,0,6/1/2017,12:15:00,PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462635,2304,theft-parts-from-vehicle,theft-from-motor-vehicle,11/30/2019 12:01:00 AM,7604 E 28TH AVE,-104.899517,39.755709,512,stapleton,1,0,11/30/2019,12:01:00,AM
462636,4104,liquor-possession,drug-alcohol,12/4/2019 8:35:00 PM,2000 BLOCK E COLFAX AVE,-104.963127,39.739432,622,cheesman-park,1,0,12/4/2019,8:35:00,PM
462637,5420,traffic-accident-dui-duid,traffic-accident,12/28/2019 8:27:00 PM,1200 BLK S QUIVAS ST,-105.006446,39.693792,422,ruby-hill,0,1,12/28/2019,8:27:00,PM
462638,5016,violation-of-restraining-order,all-other-crimes,11/16/2019 5:20:00 PM,1717 CHAMPA ST,-104.992694,39.747876,611,cbd,1,0,11/16/2019,5:20:00,PM
